# BAG3 Co-IP Watson Analysis
##### by Emir Turkes

This analysis utilizes Watson for Drug Discovery to validate Co-IP MS (co-immunoprecipitation mass spectrometry) data with BAG3 as the primary target. There are 4 goals thus far:
1. Correlation of the data with that of IBM's knowledge base to get a sense of where it stands against existing literature.
2. Clustering of the Co-IP'ed proteins into various ontological groups (disease relevance, biochemical pathways, and chemical classification come to mind)
3. Breakdown of upstream, downstream, and intermediate biochemical pathways between Co-IP'ed proteins and their clusters.
4. Confirmation of enrichment of proteins involved in endocytosis/membrane fusion event and dendritic localization/function

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*

# Copyright 2019 Emir Turkes
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""The main analysis routine."""


import os
import types

import pandas as pd

import ibm_botocore.client as ic
import ibm_boto3 as ib
import pixiedust

In [ ]:
# Read in data.    
body = client_75340283764447f99797650de21e6211.get_object(Bucket='bag3coip-donotdelete-pr-wlavvdgoryxgjb',Key='co_ip_no_keratin2.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
co_ip_df = pd.read_csv(body, na_values = ['#VALUE!', '#DIV/0!'])

# Sort by "fold of changes".
cols = ["fold of changes"]
co_ip_df.drop(0, inplace=True)
co_ip_df.dropna(axis=0, subset=cols, inplace=True)
co_ip_df[cols] = co_ip_df[cols].astype('int')
#co_ip_df.sort_values(by=cols, inplace=True, ascending=False)

# Reduce to two columns and top 10 genes.
cols = ["Unnamed: 1", "fold of changes"]
co_ip_df = co_ip_df[cols]
co_ip_df.rename(columns={"Unnamed: 1": "Genes", "fold of changes": "Fold change"}, inplace=True)
co_ip_df = co_ip_df.head(20)

# Cut string to only gene name and capitalize.
cols = "Genes"
co_ip_df[cols] = co_ip_df[cols].str.extract("((?<=GN=).*(?= PE=))", expand=True)
co_ip_df["Genes"] = co_ip_df["Genes"].str.upper()


## Replace gene names with Watson entities.
entity_set_df = pd.read_csv(body, na_values = ['#VALUE!', '#DIV/0!'])
entity_set_df.index = entity_set_df.index + 1
entity_set_df
co_ip_df["Genes"] = entity_set_df["Entity name"]

co_ip_df

### Watson Explore Networks

First, a visualization of the relationship between the top 20 genes based on literature reporting of their modulation of BAG3.

### Generate confidence scores

Watson scores the likelihood of a relationship as happening above chance through a scale ranging from 0-100.

In [ ]:
# Read in data.
relationship_graph_df = pd.read_csv(body, na_values = ['#VALUE!', '#DIV/0!'])

# Sort by "Confidence".
cols = ["Confidence"]
relationship_graph_df.drop(0, inplace=True)
relationship_graph_df.dropna(axis=0, subset=cols, inplace=True)
relationship_graph_df[cols] = relationship_graph_df[cols].astype('int')
relationship_graph_df.sort_values(by=cols, inplace=True, ascending=False)

# Only show relationships directly involving BAG3
relationship_graph_df = relationship_graph_df[
    relationship_graph_df["Source name"].str.contains('BAG3') |
    relationship_graph_df["Target name"].str.contains('BAG3')
]

# Reduce to three columns.
cols = ["Source name", "Target name", "Confidence"]
relationship_graph_df = relationship_graph_df[cols]

display(relationship_graph_df)

### Merge confidence scores with the Co-IP gene list

Rows that do not have a common gene are filled with the value "NaN".

In [ ]:
#co_ip_df = co_ip_df.assign(Confidence=co_ip_df['Genes'].isin(
#    relationship_graph_df['Target name']).astype(int)
#                          )
merge_df = co_ip_df
merge_df.at[5, "Confidence"] = 88
merge_df.at[19, "Confidence"] = 57
#co_ip_df["Confidence"] = np.where(co_ip_df["Genes"].isin(
#    relationship_graph_df['Target name']), 81, 0
#                                 )
display(merge_df)

### Reduce list to common genes only

It appears that only two highly Co-IP'ed genes could be backed up by Watson's literature mining, even without any filtering or thresholds.  
The next step is to verify that the mining run was done or interpreted properly.

In [ ]:
display(merge_df.loc[[5, 19]])